In [2]:
import json
import malaya
from glob import glob

In [3]:
language_detection = malaya.language_detection.fasttext()

Downloading:   0%|          | 0.00/31.1M [00:00<?, ?B/s]

In [4]:
language_detection.predict(['helo', 'tak suka'])

['eng', 'malay']

In [5]:
files = glob('interests/*.json')
len(files)

798

In [6]:
from tqdm import tqdm

rejected_chars = 'óÁü'
bahasa = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    for d in data:
        if d['summary']:
            if any([c in d['summary'] for c in rejected_chars]):
                continue
            
            if 'ISBN' in d['summary']:
                continue
            
            if language_detection.predict([d['summary']])[0] in ['malay', 'ind']:
                bahasa.append(d)

100%|█████████████████████████████████████████| 798/798 [00:49<00:00, 15.99it/s]


In [7]:
len(bahasa)

11445

In [8]:
rejected_chars = 'óÁü'
rejected = ['sivut', 'published', 'overview', 'review']
filtered = []
for d in tqdm(bahasa):
    if any([c in d['summary'] for c in rejected_chars]):
        continue
    
    if 'ISBN' in d['summary']:
        continue
    
    if any([r in d['summary'].lower() for r in rejected]):
        continue
        
    prob = language_detection.predict_proba([d['summary']])[0]
    if max(prob.values()) < 0.8:
        continue
        
    filtered.append(d)
    
len(filtered)

100%|██████████████████████████████████| 11445/11445 [00:00<00:00, 11507.57it/s]


6907

In [10]:
with open('filtered-search.json', 'w') as fopen:
    json.dump(filtered, fopen)

In [11]:
!mkdir pdf

mkdir: cannot create directory ‘pdf’: File exists


In [12]:
import requests
import time

cookies = {
    'from_cache': '',
    'auvid': 'MTY4MzQzNzkyMjk4NDowLjk0ODU3MDk4NDg1NDQ%3D',
    '_ga': 'GA1.2.534798164.1683437924',
    '_gid': 'GA1.2.1815999583.1683437924',
    'overridden_user_tests': '%7B%22google_account_permissions_v2%22%3A%22without_contacts%22%7D',
    'user_id': '38068128',
    'cookie_test': '38068128',
    'long_term_login': 'true',
    'login_token': '38068128%3Ba1784e11e7d9103b315e841f',
    '_sharedid': 'bdab44d3-8c83-4355-86f4-75591986fab6',
    '__qca': 'I0-518466091-1683439461266',
    '_pbjs_userid_consent_data': '3524755945110770',
    'ki_r': 'aHR0cHM6Ly9tYWxheWEuYWNhZGVtaWEuZWR1Lw%3D%3D',
    '_lr_env_src_ats': 'false',
    'pbjs-unifiedid': '%7B%22TDID%22%3A%2259a14f4e-9cd9-4559-947d-09f703acfc99%22%2C%22TDID_LOOKUP%22%3A%22TRUE%22%2C%22TDID_CREATED_AT%22%3A%222023-04-07T07%3A58%3A08%22%7D',
    'admin_unrecorded_tests': '%7B%22auth_system_version%22%3A%7B%22bucket%22%3A%22login_token_only%22%2C%22buckets%22%3A%5B%22y_cookie%22%2C%22login_token_only%22%5D%7D%2C%22localize_frontend%22%3A%7B%22bucket%22%3A%22localize%22%2C%22buckets%22%3A%5B%22control%22%2C%22localize%22%5D%7D%7D',
    'ab_tests': '%7B%22all_premium_general_modal_2023_04%22%3A%22control%22%2C%22profile_page_related_authors%22%3A%22related_authors%22%2C%22all_premium_faculty_modal_2023_04%22%3A%22faculty_image%22%7D',
    'ki_t': '1683446284799%3B1683446284799%3B1683447252094%3B1%3B9',
    'last_load_time': '1683447252',
    'cto_bundle': 'GubNql95MDE2bDVjUzF0eEJlbmwlMkI0dkxFSkUlMkYxWFJWMG9UZFZCMzU5YVBnJTJCRXFTZWhVS0ElMkY0ZjhjU3A5YlJvVGJDc29rJTJCbVJVS0UxWkYwbTdEQ3ZBRWI4dFdrcWVQSjJUdVZxUVdPejFHNVplZGxwcnF0WUpTdDlZc3pFNFc5ZlNXQXcxUm8xViUyRiUyRkh3N3lLZkhwanNBZGpCQSUzRCUzRA',
    'cto_bidid': 'Mtch719PcHRRZlpxVWhxZnlra0w1RlBvSzllUlRYYiUyQnAzMTFZMnJzQzZSd1pqZ1NnNWM5QnNsJTJCb21iUXQ0WERkeUVzTDBwMWFmNTFEem5XVkxSSUtqYyUyQmU5cm9NWldXRHdyNmwxSXFndGprQ1pkdyUzRA',
    'yauic': 'eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaVl6T0RBMk9ERXlPRHRoTVRjNE5HVXhNV1UzWkRreE1ETmlNekUxWlRnME1XWUdPZ1pGVkE9PSIsImV4cCI6IjIwMjQtMDUtMDdUMDg6MTU6MDIuNjAyWiIsInB1ciI6bnVsbH19--5e1b19b4da42a13efc7ab718556d30a13085494b',
    '_cookie_session': 'WXdhaHVCSUpidnRzRUFhR1ZDNjZxaTlnVGtTeVhsT2c1WGtvR0tqUHZHQTIrS3k0TU1yQTF0a0Z4SCsxb0ZXUUhaSy83aFcvUkI1UjhVdmZ1b0VFWDBWd2hhRkM3Z1IvWVN0NFZrQkdtV1VzaDJvdHJtdXFycHVVUGY1bVFPeGIvcG1MM3dua3pkOFN1cEFoTE5Ub3VWdU9IVzV4aTVNbFVWYVRjdWdBMThjRTMyRXlsZVhGTlBWT0VJUVFCRFVST1NtVmhvNUQ0TDJldWlSQm1zYzd3ODRYczR1QUVoaGJSenJXTG82RGlwWUg2T3krektybnA0WHc1eFR5TWtzaTlyZlJPRm9DZkJZSDgrTjFqRjV5QUJ4bFIyQ0E3YWZGRk9JenEvSU95MlZleU4ybTFsVEZhTzBIZ0gvaWM0UWR2eUlvY1dJRE1wbGhtdFUyM1A1bjE5dlovV3NObHlyQk16eFlMR09ITEozVWN2bnM5QnVsc2JodDYraVB4eGM2NEZ4ZWViWWZWWXdQekdsMzBPUGNOK0QzQ0pKMEVhY0NGZXJHdGhXZTcyQnI0dG8wMDJjZ0FDTzV3U25ZL2VWa2RrRlh4Z2UyQXQ2UnJaUkNZZ05TeW1hQmhkdU1qUWxPVklnaTNaa1RrRnM9LS1vOTVoZkRLN0hndlVqYTB0dDRsekJRPT0%3D--ba9351aec1797d1e6f9bb70695ed88daa246b18e',
}

headers = {
    'authority': 'apiitmalaysia.academia.edu',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': 'from_cache=; auvid=MTY4MzQzNzkyMjk4NDowLjk0ODU3MDk4NDg1NDQ%3D; _ga=GA1.2.534798164.1683437924; _gid=GA1.2.1815999583.1683437924; overridden_user_tests=%7B%22google_account_permissions_v2%22%3A%22without_contacts%22%7D; user_id=38068128; cookie_test=38068128; long_term_login=true; login_token=38068128%3Ba1784e11e7d9103b315e841f; _sharedid=bdab44d3-8c83-4355-86f4-75591986fab6; __qca=I0-518466091-1683439461266; _pbjs_userid_consent_data=3524755945110770; ki_r=aHR0cHM6Ly9tYWxheWEuYWNhZGVtaWEuZWR1Lw%3D%3D; _lr_env_src_ats=false; pbjs-unifiedid=%7B%22TDID%22%3A%2259a14f4e-9cd9-4559-947d-09f703acfc99%22%2C%22TDID_LOOKUP%22%3A%22TRUE%22%2C%22TDID_CREATED_AT%22%3A%222023-04-07T07%3A58%3A08%22%7D; admin_unrecorded_tests=%7B%22auth_system_version%22%3A%7B%22bucket%22%3A%22login_token_only%22%2C%22buckets%22%3A%5B%22y_cookie%22%2C%22login_token_only%22%5D%7D%2C%22localize_frontend%22%3A%7B%22bucket%22%3A%22localize%22%2C%22buckets%22%3A%5B%22control%22%2C%22localize%22%5D%7D%7D; ab_tests=%7B%22all_premium_general_modal_2023_04%22%3A%22control%22%2C%22profile_page_related_authors%22%3A%22related_authors%22%2C%22all_premium_faculty_modal_2023_04%22%3A%22faculty_image%22%7D; ki_t=1683446284799%3B1683446284799%3B1683447252094%3B1%3B9; last_load_time=1683447252; cto_bundle=GubNql95MDE2bDVjUzF0eEJlbmwlMkI0dkxFSkUlMkYxWFJWMG9UZFZCMzU5YVBnJTJCRXFTZWhVS0ElMkY0ZjhjU3A5YlJvVGJDc29rJTJCbVJVS0UxWkYwbTdEQ3ZBRWI4dFdrcWVQSjJUdVZxUVdPejFHNVplZGxwcnF0WUpTdDlZc3pFNFc5ZlNXQXcxUm8xViUyRiUyRkh3N3lLZkhwanNBZGpCQSUzRCUzRA; cto_bidid=Mtch719PcHRRZlpxVWhxZnlra0w1RlBvSzllUlRYYiUyQnAzMTFZMnJzQzZSd1pqZ1NnNWM5QnNsJTJCb21iUXQ0WERkeUVzTDBwMWFmNTFEem5XVkxSSUtqYyUyQmU5cm9NWldXRHdyNmwxSXFndGprQ1pkdyUzRA; yauic=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaVl6T0RBMk9ERXlPRHRoTVRjNE5HVXhNV1UzWkRreE1ETmlNekUxWlRnME1XWUdPZ1pGVkE9PSIsImV4cCI6IjIwMjQtMDUtMDdUMDg6MTU6MDIuNjAyWiIsInB1ciI6bnVsbH19--5e1b19b4da42a13efc7ab718556d30a13085494b; _cookie_session=WXdhaHVCSUpidnRzRUFhR1ZDNjZxaTlnVGtTeVhsT2c1WGtvR0tqUHZHQTIrS3k0TU1yQTF0a0Z4SCsxb0ZXUUhaSy83aFcvUkI1UjhVdmZ1b0VFWDBWd2hhRkM3Z1IvWVN0NFZrQkdtV1VzaDJvdHJtdXFycHVVUGY1bVFPeGIvcG1MM3dua3pkOFN1cEFoTE5Ub3VWdU9IVzV4aTVNbFVWYVRjdWdBMThjRTMyRXlsZVhGTlBWT0VJUVFCRFVST1NtVmhvNUQ0TDJldWlSQm1zYzd3ODRYczR1QUVoaGJSenJXTG82RGlwWUg2T3krektybnA0WHc1eFR5TWtzaTlyZlJPRm9DZkJZSDgrTjFqRjV5QUJ4bFIyQ0E3YWZGRk9JenEvSU95MlZleU4ybTFsVEZhTzBIZ0gvaWM0UWR2eUlvY1dJRE1wbGhtdFUyM1A1bjE5dlovV3NObHlyQk16eFlMR09ITEozVWN2bnM5QnVsc2JodDYraVB4eGM2NEZ4ZWViWWZWWXdQekdsMzBPUGNOK0QzQ0pKMEVhY0NGZXJHdGhXZTcyQnI0dG8wMDJjZ0FDTzV3U25ZL2VWa2RrRlh4Z2UyQXQ2UnJaUkNZZ05TeW1hQmhkdU1qUWxPVklnaTNaa1RrRnM9LS1vOTVoZkRLN0hndlVqYTB0dDRsekJRPT0%3D--ba9351aec1797d1e6f9bb70695ed88daa246b18e',
    'pragma': 'no-cache',
    'referer': 'https://apiitmalaysia.academia.edu/',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

# response = requests.get(
#     'https://apiitmalaysia.academia.edu/Departments/Accounting_and_Finance/Documents?page=1',
#     cookies=cookies,
#     headers=headers,
# )

In [13]:
def is_downloadable(url):
    h = requests.head(url, cookies=cookies,
    headers=headers,allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return h

def download_file_cloud(url, filename):
    r = requests.get(url, cookies=cookies,
    headers=headers,stream=True)
    total_size = int(r.headers['content-length'])
    version = int(r.headers.get('X-Bz-Upload-Timestamp', 0))
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as f:
        for data in r.iter_content(chunk_size=1_048_576):
            f.write(data)
    return version

In [14]:
import os
import random

def exist(filename):
    splitted = [os.path.split(f)[1].split('_')[0] for f in glob('pdf/*')]
    return str(filename) in splitted

In [ ]:
for i in tqdm(range(len(filtered))):
    try:
        id_ = filtered[i]['downloadable_attachments'][0]['id']
        if exist(id_):
            continue
        url = f'https://www.academia.edu/attachments/{id_}/download_file'
        h = is_downloadable(url)
        filename = h.headers['Content-Disposition'].split('filename=')[1]
        filename = f'{id_}_{filename}'
        filename = os.path.join('pdf', filename)
        if not os.path.exists(filename):
            download_file_cloud(url, filename)
            filtered[i]['filename'] = filename

        time.sleep(random.uniform(20.0, 120.0))
    except Exception as e:
        print(e)

  2%|▊                                   | 148/6907 [37:49<106:06:05, 56.51s/it]

'bool' object has no attribute 'headers'


  2%|▊                                    | 149/6907 [37:50<74:34:04, 39.72s/it]

'bool' object has no attribute 'headers'


  2%|▊                                    | 150/6907 [37:51<52:33:16, 28.00s/it]

...

 55%|███████████████████▏               | 3796/6907 [1:26:05<1:21:13,  1.57s/it]

'bool' object has no attribute 'headers'


 55%|███████████████████▏               | 3797/6907 [1:26:07<1:20:25,  1.55s/it]

'bool' object has no attribute 'headers'


 55%|███████████████████▏               | 3798/6907 [1:26:08<1:05:53,  1.27s/it]